<center>
<h1> TP-Projet d'optimisation numérique </h1>
<h1> Algorithme du Lagrangien Augmenté </h1>
</center>

## Implémentation

1. Implémenter l'algorithme du lagrangien augmenté, en utilisant les différentes méthodes
qui ont été vues en première partie pour la résolution de la suite de problémes sans
contraintes (fichier `Lagrangien_Augmente.jl`). La spécification de l'algorithme du Lagrangien augmenté est donnée ci-dessous.
 

In [11]:
using LinearAlgebra
#using Documenter
using Markdown  
include("Lagrangien_Augmente.jl")
@doc Lagrangien_Augmente

#### Objet

Résolution des problèmes de minimisation avec une contrainte d'égalité scalaire par l'algorithme du lagrangien augmenté.

#### Syntaxe

```julia
xmin,fxmin,flag,iter,muks,lambdaks = Lagrangien_Augmente(algo,f,gradf,hessf,c,gradc,hessc,x0,options)
```

#### Entrées

  * algo : (String) l'algorithme sans contraintes à utiliser:

      * "newton"  : pour l'algorithme de Newton
      * "cauchy"  : pour le pas de Cauchy
      * "gct"     : pour le gradient conjugué tronqué
  * f : (Function) la fonction à minimiser
  * gradf       : (Function) le gradient de la fonction
  * hessf       : (Function) la hessienne de la fonction
  * c     : (Function) la contrainte [x est dans le domaine des contraintes ssi $c(x)=0$]
  * gradc : (Function) le gradient de la contrainte
  * hessc : (Function) la hessienne de la contrainte
  * x0 : (Array{Float,1}) la première composante du point de départ du Lagrangien
  * options : (Array{Float,1})

    1. epsilon     : utilisé dans les critères d'arrêt
    2. tol         : la tolérance utilisée dans les critères d'arrêt
    3. itermax     : nombre maximal d'itération dans la boucle principale
    4. lambda0     : la deuxième composante du point de départ du Lagrangien
    5. mu0, tho    : valeurs initiales des variables de l'algorithme

#### Sorties

  * xmin : (Array{Float,1}) une approximation de la solution du problème avec contraintes
  * fxmin : (Float) $f(x_{min})$
  * flag : (Integer) indicateur du déroulement de l'algorithme

      * 0    : convergence
      * 1    : nombre maximal d'itération atteint
      * (-1) : une erreur s'est produite
  * niters : (Integer) nombre d'itérations réalisées
  * muks : (Array{Float64,1}) tableau des valeurs prises par mu_k au cours de l'exécution
  * lambdaks : (Array{Float64,1}) tableau des valeurs prises par lambda_k au cours de l'exécution

#### Exemple d'appel

```julia
using LinearAlgebra
algo = "gct" # ou newton|gct
f(x)=100*(x[2]-x[1]^2)^2+(1-x[1])^2
gradf(x)=[-400*x[1]*(x[2]-x[1]^2)-2*(1-x[1]) ; 200*(x[2]-x[1]^2)]
hessf(x)=[-400*(x[2]-3*x[1]^2)+2  -400*x[1];-400*x[1]  200]
c(x) =  (x[1]^2) + (x[2]^2) -1.5
gradc(x) = [2*x[1] ;2*x[2]]
hessc(x) = [2 0;0 2]
x0 = [1; 0]
options = []
xmin,fxmin,flag,iter,muks,lambdaks = Lagrangien_Augmente(algo,f,gradf,hessf,c,gradc,hessc,x0,options)
```

#### Tolérances des algorithmes appelés

Pour les tolérances définies dans les algorithmes appelés (Newton et régions de confiance), prendre les tolérances par défaut définies dans ces algorithmes.


3. Vérifier que les tests ci-dessous passent.

In [12]:
using Test

# Tolérance pour les tests d'égalité
tol_erreur = sqrt(eps())

## ajouter les fonctions de test
include("../test/fonctions_de_tests.jl")
include("../test/tester_lagrangien_augmente.jl")
include("../src/Algorithme_De_Newton.jl")
include("../src/Pas_De_Cauchy.jl")
include("../src/Gradient_Conjugue_Tronque.jl")
include("../src/Regions_De_Confiance.jl")
include("../src/Lagrangien_Augmente.jl")

affiche = false

@testset "Test lagrangien augmente" begin
	tester_lagrangien_augmente(affiche, Lagrangien_Augmente)
end;



Test Summary:            | Pass  Total  Time
Test lagrangien augmente |   12     12  2.8s


## Interprétation

 1. Commenter les résultats obtenus, en étudiant notamment les valeurs de $\lambda_k$ et $\mu_k$.
 
 2. Étudier l'influence du paramètre $\tau$ dans la performance de l'algorithme. Pour cela Vous réaliserez des tests numériques.
 
 3. **Supplémentaire** : 
      Que proposez-vous comme méthode pour la résolution des problèmes avec
      des contraintes à la fois d'égalité et d'inégalité ? Implémenter (si le temps le permet)
      ce nouvel algorithme.

In [13]:
#using Pkg; Pkg.add("LinearAlgebra"); Pkg.add("Markdown")
# using Documenter
using LinearAlgebra
using Markdown                             # Pour que les docstrings en début des fonctions ne posent
                                           # pas de soucis. Ces docstrings sont utiles pour générer 
                                           # la documentation sous GitHub
include("Lagrangien_Augmente.jl")
include("Regions_De_Confiance.jl")
include("Pas_De_Cauchy.jl")
include("Gradient_Conjugue_Tronque.jl")

# Affichage les sorties de l'algorithme des Régions de confiance
function my_afficher_resultats(algo,nom_fct,point_init,xmin,fxmin,flag,sol_exacte,nbiters)
	println("-------------------------------------------------------------------------")
	printstyled("Résultats de : ",algo, " appliqué à ",nom_fct, " au point initial ", point_init, ":\n",bold=true,color=:blue)
	println("  * xsol = ",xmin)
	println("  * f(xsol) = ",fxmin)
	println("  * nb_iters = ",nbiters)
	println("  * flag = ",flag)
	println("  * sol_exacte : ", sol_exacte)
end

fct1(x) =  2*(x[1]+x[2]+x[3]-3)^2 + (x[1]-x[2])^2 + (x[2]-x[3])^2
# la gradient de la fonction fct1
function grad_fct1(x)
    y1 = 4*(x[1]+x[2]+x[3]-3) + 2*(x[1]-x[2])
    y2 = 4*(x[1]+x[2]+x[3]-3) - 2*(x[1]-x[2]) +2*(x[2]-x[3])
    y3 = 4*(x[1]+x[2]+x[3]-3) - 2*(x[2]-x[3])
    return [y1;y2;y3]
end
# la hessienne de la fonction fct1
    hess_fct1(x) = [6 2 4;2 8 2;4 2 6]
    contrainte1(x) =  x[1]+x[3]-1
    grad_contrainte1(x) = [1 ;0; 1]
    hess_contrainte1(x) = [0 0 0;0 0 0;0 0 0]
    sol_exacte = [0.5 ; 1.25 ; 0.5]
    x0 = [0; -1; 1]

    ############### Influence de mu #################
    print("Etudions l'influence de mu : ")
    options=[1e-2, 1e-5, 1000,2,100, 2]
    xmin, f_min, flag, nb_iters, muks, lambdaks = Lagrangien_Augmente("cauchy", fct1, contrainte1, grad_fct1, hess_fct1, grad_contrainte1, hess_contrainte1, x0, options)
    my_afficher_resultats("Lagrangien_Augmente_gct", "f1", x0, xmin, f_min, flag, sol_exacte, nb_iters)
    print("les valeurs prises par lambda : ")
    print(lambdaks)
    print("\n les valeurs prises par mu : ")
    print(muks)

    options=[1e-2, 1e-5, 1000,2,400, 2]
    xmin, f_min, flag, nb_iters, muks, lambdaks = Lagrangien_Augmente("cauchy", fct1, contrainte1, grad_fct1, hess_fct1, grad_contrainte1, hess_contrainte1, x0, options)
    my_afficher_resultats("Lagrangien_Augmente_gct", "f1", x0, xmin, f_min, flag, sol_exacte, nb_iters)
    print("les valeurs prises par lambda : ")
    print(lambdaks)
    print("\n les valeurs prises par mu : ")
    print(muks)
     
     ############### Influence de lambda #################
     print("\n Etudions l'influence de lambda : ")
     options=[1e-2, 1e-5, 1000,2,100, 2]
     xmin, f_min, flag, nb_iters, muks, lambdaks = Lagrangien_Augmente("gct", fct1, contrainte1, grad_fct1, hess_fct1, grad_contrainte1, hess_contrainte1, x0, options)
     my_afficher_resultats("Lagrangien_Augmente_gct", "f1", x0, xmin, f_min, flag, sol_exacte, nb_iters)
     print("les valeurs prises par lambda : ")
     print(lambdaks)
     print("\n les valeurs prises par mu : ")
     print(muks)
 
     options=[1e-2, 1e-5, 1000,4,100, 2]
     xmin, f_min, flag, nb_iters, muks, lambdaks = Lagrangien_Augmente("gct", fct1, contrainte1, grad_fct1, hess_fct1, grad_contrainte1, hess_contrainte1, x0, options)
     my_afficher_resultats("Lagrangien_Augmente_gct", "f1", x0, xmin, f_min, flag, sol_exacte, nb_iters)
     print("les valeurs prises par lambda : ")
     print(lambdaks)
     print("\n les valeurs prises par mu : ")
     print(muks)
    ############### Influence de tho #################
    print("\n Etudions l'influence de tho : ")
    options=[1e-2, 1e-5, 1000,2,100, 1]
    xmin, f_min, flag, nb_iters= Lagrangien_Augmente("gct", fct1, contrainte1, grad_fct1, hess_fct1, grad_contrainte1, hess_contrainte1, x0, options)
    my_afficher_resultats("Lagrangien_Augmente_gct", "f1", x0, xmin, f_min, flag, sol_exacte, nb_iters)

    options=[1e-2, 1e-5, 1000,2,100, 6]
    xmin, f_min, flag, nb_iters= Lagrangien_Augmente("gct", fct1, contrainte1, grad_fct1, hess_fct1, grad_contrainte1, hess_contrainte1, x0, options)
    my_afficher_resultats("Lagrangien_Augmente_gct", "f1", x0, xmin, f_min, flag, sol_exacte, nb_iters)

    
    

Etudions l'influence de mu : -------------------------------------------------------------------------
Résultats de : Lagrangien_Augmente_gct appliqué à f1 au point initial [0, -1, 1]:
  * xsol = [0.49999770042217423, 1.2500000901856034, 0.5000028178082356]
  * f(xsol) = 2.249997667977046
  * nb_iters = 5
  * flag = 0
  * sol_exacte : [0.5, 1.25, 0.5]
les valeurs prises par lambda : [2.0, 4.392303019236142, 4.495332581028542, 4.499903671439263]
 les valeurs prises par mu : [100.0, 200.0]-------------------------------------------------------------------------
Résultats de : Lagrangien_Augmente_gct appliqué à f1 au point initial [0, -1, 1]:
  * xsol = [0.499993377165322, 1.2500001962438476, 0.5000073693598335]
  * f(xsol) = 2.249996640736531
  * nb_iters = 3
  * flag = 0
  * sol_exacte : [0.5, 1.25, 0.5]
les valeurs prises par lambda : [2.0, 4.472107638403287, 4.499692962996747]
 les valeurs prises par mu : [400.0]
 Etudions l'influence de lambda : --------------------------------------

Réponses : 

1- On constate que l'augmentation de Mu réduit le nombre d'itération, et permet de converger plus rapidement vers la solution si elle existe , chose qui est montrée au dessus. Mais, cette approche reste moins performante par rapport aux approches précedentes qui assurent une convergence plus rapide. 
On remarque également que les valeurs de lambda et mu augmentent en sortie de l'algorithme, cela montre que la solution du problème sans contrainte ne satisfait pas les contraintes appliquées ce qui nous pousse à modifier les paramètres de pénalité plusieurs fois.


2- Ainsi, on remarque qu'avoir une valeur de 'tho' très faible assure une convergence en un nombre d'itération faible mais loin de la solution exacte. Contrairement à des valeurs plus significatives qui assurent en meme temps une convergence plus ou moins rapide et une solution plus proche de la solution exacte. Ainsi, il faut faire un compromis entre la certitidue de la solution et la rapidité de l'algorithme .